<h1>Dataset dos apartamentos</h1>
<h2>Introdução</h2>
<font size=3>
O dataset será construído retirando os dados do <a href='https://www.quintoandar.com.br/'>Quinto Andar</a> e juntando com outros dados externos para construção de novas features.
</font>
<h2>Índice</h2>
<font size=3>
<ol>
    <li><a href='#5A'>Pegando dos dados do 5A</a></li>
    <li><a href='#METRO'>Dados das estações do mêtro</a></li>
    <li><a href='#NEW'>Criando novas features</a></li>
    <li><a href='#SAVE'>Salvando</a></li>
</ol>
</font>

In [26]:
import pandas as pd
import numpy as np
import requests as rq
import json
from time import sleep
import warnings
from geopy import distance
from datetime import date

pd.options.display.max_rows = 999
warnings.filterwarnings("ignore")

<a name='5A'></a>
<h2>Pegando os dados do 5A</h2>
<h3>Coordenadas dos bairros de São Paulo</h3>
<font size=3>
Vamos criar um função para pegar os coordenadas dos bairros de São Paulo para utilizarmos para quando formos fazer request no 5A.
</font>

In [27]:
def get_coordinates():
    df = pd.read_csv('district.csv')
    latlon = []
    for key, value in df.iterrows():
        diff = 0.008
        lat = value['Latitude']
        lon = value['Longitude']
        latlon.append([str(round(lat-diff,2))+','+str(round(lon+diff,2)),str(round(lat+diff,2))+','+str(round(lon-diff,2))])    
    df = df.merge(pd.DataFrame(latlon), how='left', left_index=True, right_index=True)
    df.columns = ['District', 'Latitude', 'Longitude', 'Coord_0', 'Coord_1']	
    return df

<font size=3>
Agora vamos definir uma função que irá requisitar os apartamentos disponíveis nas coordenadas de um bairro específico.
</font>

In [28]:
def get_district(coord_0, coord_1):
    url = 'https://www.quintoandar.com.br/api/yellow-pages/search?q=(and%20tipo:%27Apartamento%27(and%20(and%20for_rent:%27true%27)))&fq=local:[%27{}%27,%27{}%27]&return=id,local,aluguel,area,quartos,custo,endereco,regiao_nome,special_conditions,listing_tags,tipo,promotions,condo_iptu,vagas,amenidades&size=1000&q.parser=structured'
    furl = url.format(coord_1, coord_0)
    response = rq.get(furl).json()
    apes = []
    for hit in response['hits']['hit']:
        ap_id = hit.get('id')
        quartos = hit.get('fields').get('quartos')
        area = hit.get('fields').get('area')
        custo = hit.get('fields').get('custo')
        vagas = hit.get('fields').get('vagas')
        amenidades = hit.get('fields').get('amenidades')
        if 'NaoMobiliado' in amenidades:
            mobiliado = 0
        else:
            mobiliado = 1
        local = hit.get('fields').get('local')
        bairro = hit.get('fields').get('regiao_nome')
        endereco = hit.get('fields').get('endereco')
        apes.append([ap_id, quartos, area, custo, vagas, mobiliado, local, bairro, endereco])
    apes = pd.DataFrame(apes, columns=['id', 'quartos', 'area', 'custo', 'vagas', 'mobiliado',
                                       'local', 'bairro', 'endereco'])
    return apes

<font size=3>
Com as funções feitas, vamos criar a última que irá pegar todos os apartamentos da cidade de São Paulo.
</font>

In [29]:
def get_apartments(district):
    apartments = pd.DataFrame()
    for key, value in district.iterrows():
        apartments = pd.concat([apartments, get_district(value['Coord_0'], value['Coord_1'])], ignore_index=True)
        sleep(1)
    return apartments

<font size=3>
Montando o dataframe com as informações dos apartamentos.
</font>

In [30]:
district = get_coordinates()
apartments = get_apartments(district)
apartments.drop_duplicates(inplace=True)
apartments.shape

(4764, 9)

In [31]:
apartments['latitude'] = apartments['local'].apply(lambda x: x.split(',')[0])
apartments['longitude'] = apartments['local'].apply(lambda x: x.split(',')[1])
apartments.drop('local', axis=1, inplace=True)

In [32]:
apartments.tail()

,id,quartos,area,custo,vagas,mobiliado,bairro,endereco,latitude,longitude
5691,893080919,2,68,1902,1,0,Jardim Anália Franco,Rua Atalaia Velha,-23.5657732,-46.5713469
5692,893094660,1,50,1929,1,0,Jardim Anália Franco,Avenida Sapopemba,-23.5643634,-46.5707749
5693,893015276,3,113,6411,2,0,Vila Invernada,Rua Amapá,-23.5698236,-46.5717586
5694,893026105,2,93,4068,2,0,Vila Invernada,Rua Amapá,-23.5698236,-46.5717586
5695,892973029,1,55,1114,0,0,Mooca,Rua Piraçununga,-23.5614488,-46.5771146


<a name='METRO'></a>
<h2>Dados das estações do mêtro</h2>

In [33]:
metro = pd.read_csv('metrosp_stations_v2.csv').drop('Unnamed: 0', axis=1)
metro.head()

,name,station,lat,lon,line
0,Aacd Servidor,aacd-servidor,-23.597825,-46.652374,['lilas']
1,Adolfo Pinheiro,adolfo-pinheiro,-23.650073,-46.704206,['lilas']
2,Alto Da Boa Vista,alto-da-boa-vista,-23.641625,-46.699434,['lilas']
3,Alto Do Ipiranga,alto-do-ipiranga,-23.602237,-46.612486,['verde']
4,Ana Rosa,ana-rosa,-23.581871,-46.638104,"['azul', 'verde']"


<a name='NEW'></a>
<h2>Criando novas features</h2>
<font size=3>
Para adicionar mais informações para aumentar a performance do nosso futuro modelo vamos criar mais features para ajudar.<br><br>
<li>Estação mais próxima e a distância</li>
</font>

In [34]:
def distancia(lat, lon):
    dist = []
    for idx, value in metro.iterrows():
        dist.append(distance.distance((metro.iloc[idx]['lat'],metro.iloc[idx]['lon']), (lat,lon)).km)
    min_dist = min(dist)
    estacao = metro.iloc[dist.index(min_dist)]['name']
    return min_dist, estacao

In [35]:
apartments['metros'] = apartments.apply(lambda x: distancia(x['latitude'], x['longitude']), axis=1)
apartments['estacao'] = apartments['metros'].apply(lambda x: x[1])
apartments['distancia'] = apartments['metros'].apply(lambda x: x[0])
apartments['distancia'] = apartments['distancia'].apply(lambda x: round(x, 2))
apartments.drop('metros', axis=1, inplace=True)
apartments.head()

,id,quartos,area,custo,vagas,mobiliado,bairro,endereco,latitude,longitude,estacao,distancia
0,892870723,3,200,8702,2,1,Alto de Pinheiros,Avenida Padre Pereira de Andrade,-23.5418286,-46.7188995,Vila Madalena,2.882758
1,893092681,2,125,8130,2,0,Alto de Pinheiros,Avenida Arruda Botelho,-23.5526504,-46.7195037,Pinheiros,2.413573
2,892842198,3,105,4509,2,0,Alto da Lapa,Avenida Diogenes Ribeiro de Lima,-23.5407255,-46.7172978,Vila Madalena,2.747850
3,893099931,3,77,3545,0,1,Alto de Pinheiros,Avenida Diógenes Ribeiro de Lima,-23.5424081,-46.7166143,Vila Madalena,2.641777
4,893099616,3,63,4991,0,1,Alto de Pinheiros,Avenida Diógenes Ribeiro de Lima,-23.5424081,-46.7166143,Vila Madalena,2.641777


<font size=3>
<br>
<li>Linha da estação mais próxima</li>
Vamos juntar com o dataframe do mêtro para pegar a linha da estação mais próxima.
</font>

In [42]:
apartments = apartments.merge(metro[['name', 'line']], how='left', left_on='estacao', right_on='name').drop('name', axis=1)
apartments.tail()

,id,quartos,area,custo,vagas,mobiliado,bairro,endereco,latitude,longitude,estacao,distancia,line
4759,893080919,2,68,1902,1,0,Jardim Anália Franco,Rua Atalaia Velha,-23.5657732,-46.5713469,Oratorio,2.06,['prata']
4760,893094660,1,50,1929,1,0,Jardim Anália Franco,Avenida Sapopemba,-23.5643634,-46.5707749,Oratorio,2.17,['prata']
4761,893015276,3,113,6411,2,0,Vila Invernada,Rua Amapá,-23.5698236,-46.5717586,Oratorio,1.70,['prata']
4762,893026105,2,93,4068,2,0,Vila Invernada,Rua Amapá,-23.5698236,-46.5717586,Oratorio,1.70,['prata']
4763,892973029,1,55,1114,0,0,Mooca,Rua Piraçununga,-23.5614488,-46.5771146,Tatuape,2.35,['vermelha']


<font size=3>
Primeiro é necessário ajustar o campo <i>line</i> para criar um nova feature para cada linha, depois devemos criar uma função que criará a feature de cada linha.
</font>

In [43]:
apartments['line'] = apartments['line'].apply(lambda x: x[1:-1])
apartments['line'] = apartments['line'].apply(lambda x: x.replace("'", ''))

In [44]:
def get_subway_line(linha):
    lista = []
    for idx, value in apartments.iterrows():
        tmp = value['line'].split(', ')
        if linha in tmp:
            lista.append(1)
        else:
            lista.append(0)
    return lista

In [45]:
linhas = ['amarela', 'azul', 'lilas', 'prata', 'verde', 'vermelha']

In [46]:
for linha in linhas:
    apartments['linha_{}'.format(linha)] = get_subway_line(linha)
apartments.shape

(4764, 19)

<font size=3>
Como a variável <i>line</i> já foi utilizada vamos tirá-la do dataset.
</font>

In [47]:
apartments.drop('line', axis=1, inplace=True)

<font size=3>
    <li>update_time</li>
    Vamos adicionar a variável <i>update_time</i> que não servirá para o modelo, mas apenas para termos noção de quando foi atualizado o dataset.
</font>

In [48]:
apartments['update_time'] = date.today()
apartments.tail()

,id,quartos,area,custo,vagas,mobiliado,bairro,endereco,latitude,longitude,estacao,distancia,linha_amarela,linha_azul,linha_lilas,linha_prata,linha_verde,linha_vermelha,update_time
4759,893080919,2,68,1902,1,0,Jardim Anália Franco,Rua Atalaia Velha,-23.5657732,-46.5713469,Oratorio,2.06,0,0,0,1,0,0,2020-07-19
4760,893094660,1,50,1929,1,0,Jardim Anália Franco,Avenida Sapopemba,-23.5643634,-46.5707749,Oratorio,2.17,0,0,0,1,0,0,2020-07-19
4761,893015276,3,113,6411,2,0,Vila Invernada,Rua Amapá,-23.5698236,-46.5717586,Oratorio,1.70,0,0,0,1,0,0,2020-07-19
4762,893026105,2,93,4068,2,0,Vila Invernada,Rua Amapá,-23.5698236,-46.5717586,Oratorio,1.70,0,0,0,1,0,0,2020-07-19
4763,892973029,1,55,1114,0,0,Mooca,Rua Piraçununga,-23.5614488,-46.5771146,Tatuape,2.35,0,0,0,0,0,1,2020-07-19


<font size=3>
<li>apartment_link</li>
Será um link para visualizar os apartamentos no site, e será utilizado apenas quando formos classificar quais os apartamentos seriam uma boa recomendação ou não.
</font>

In [49]:
site = 'https://www.quintoandar.com.br/imovel/'
apartments['apartment_link'] = apartments.apply(lambda x: site+x['id'], axis=1)

<font size=3>
<li>y</li>
Vamos deixar tudo como 0, ai caso seja positivo alterarmos.
</font>

In [50]:
apartments['y'] = 0

In [57]:
apartments = apartments[['id', 'y', 'quartos', 'area', 'custo', 'vagas', 'mobiliado', 'bairro',
                         'endereco', 'latitude', 'longitude', 'estacao', 'distancia',
                         'linha_amarela', 'linha_azul', 'linha_lilas', 'linha_prata',
                         'linha_verde', 'linha_vermelha', 'update_time', 'apartment_link']]

<a name='SAVE'></a>
<h2>Salvando</h2>

In [58]:
apartments.to_csv('apartamentos.csv', index=False)